In [56]:
import pandas as pd
import re

excel_url = "C:\\Users\\ymg08\\Desktop\\python_code\\food_menu_refer\\new_rcp.xlsx"
data = pd.read_excel(excel_url)
data.head()

,Unnamed: 0,RCP_NM,RCP_PARTS_DTLS,INFO_ENG,INFO_CAR,INFO_PRO,INFO_FAT
0,0,새우 두부 계란찜,"새우두부계란찜\n연두부 75g(3/4모), 칵테일새우 20g(5마리), 달걀 30g...",220.0,3.0,14.0,17.0
1,1,부추 콩가루 찜,"[1인분]조선부추 50g, 날콩가루 7g(1⅓작은술)\n·양념장 : 저염간장 3g(...",215.0,20.0,14.0,9.0
2,2,방울토마토 소박이,"●방울토마토 소박이 : \n방울토마토 150g(5개), 양파 10g(3×1cm), ...",45.0,9.0,2.0,1.0
3,3,순두부 사과 소스 오이무침,"●오이무침 :\n오이 70g(1/3개), 다진 땅콩 10g(1큰술)\n●순두부사과 ...",75.0,10.0,4.0,2.0
4,4,사과 새우 북엇국,"북엇국\n북어채 25g(15개), 새우 10g(3마리), 사과 30g(1/5개), ...",65.0,2.0,12.0,1.0


In [3]:
all_rcp = data['RCP_PARTS_DTLS']
all_rcp

0      새우두부계란찜\n연두부 75g(3/4모), 칵테일새우 20g(5마리), 달걀 30g...
1      [1인분]조선부추 50g, 날콩가루 7g(1⅓작은술)\n·양념장 : 저염간장 3g(...
2      ●방울토마토 소박이 : \n방울토마토 150g(5개), 양파 10g(3×1cm), ...
3      ●오이무침 :\n오이 70g(1/3개), 다진 땅콩 10g(1큰술)\n●순두부사과 ...
4      북엇국\n북어채 25g(15개), 새우 10g(3마리), 사과 30g(1/5개), ...
                             ...                        
995    당근(10g), 시금치(10g), 사과(10g),\n호두(5g), 크랜베리(5g),...
996    수삼 10g, 배 20g, 오이 20g, 샐러리 20g >[소스소개] 저나트륨내채소...
997    해선장 2g, 간장 1.3g, 녹말 5g, 참기름 2.5g, 식용유 5g, 마늘다진...
998    양배추 16g, 깻잎 6g, 설탕 2g, 청고추 1g, 홍고추 1g [소스소개]피클...
999    얼갈이 배추 60g, 홍고추 0.6g [소스소개저나트륨 무침양념:두부 30g, 된장...
Name: RCP_PARTS_DTLS, Length: 1000, dtype: object

In [37]:
# 딕셔너리로 변경
data_dict = data.to_dict('records')
data_dict[:3]

[{'Unnamed: 0': 0,
  'RCP_NM': '새우 두부 계란찜',
  'RCP_PARTS_DTLS': '새우두부계란찜\n연두부 75g(3/4모), 칵테일새우 20g(5마리), 달걀 30g(1/2개), 생크림 13g(1큰술), 설탕 5g(1작은술), 무염버터 5g(1작은술)\n고명\n시금치 10g(3줄기)',
  'INFO_ENG': 220.0,
  'INFO_CAR': 3.0,
  'INFO_PRO': 14.0,
  'INFO_FAT': 17.0},
 {'Unnamed: 0': 1,
  'RCP_NM': '부추 콩가루 찜',
  'RCP_PARTS_DTLS': '[1인분]조선부추 50g, 날콩가루 7g(1⅓작은술)\n·양념장 : 저염간장 3g(2/3작은술), 다진 대파 5g(1작은술), 다진 마늘 2g(1/2쪽), 고춧가루 2g(1/3작은술), 요리당 2g(1/3작은술), 참기름 2g(1/3작은술), 참깨 약간',
  'INFO_ENG': 215.0,
  'INFO_CAR': 20.0,
  'INFO_PRO': 14.0,
  'INFO_FAT': 9.0},
 {'Unnamed: 0': 2,
  'RCP_NM': '방울토마토 소박이',
  'RCP_PARTS_DTLS': '●방울토마토 소박이 : \n방울토마토 150g(5개), 양파 10g(3×1cm), 부추 10g(5줄기)\n●양념장 : \n고춧가루 4g(1작은술), 멸치액젓 3g(2/3작은술), 다진 마늘 2.5g(1/2쪽), 매실액 2g(1/3작은술), 설탕 2g(1/3작은술), 물 2ml(1/3작은술), 통깨 약간',
  'INFO_ENG': 45.0,
  'INFO_CAR': 9.0,
  'INFO_PRO': 2.0,
  'INFO_FAT': 1.0}]

In [57]:
# 재료(양g), ... 이런 식으로 된 거 분류
def extract_ingredients(recipe):
    parts = recipe['RCP_PARTS_DTLS']
    
    # 정규 표현식을 사용하여 재료와 양 튜플로 추출
    # ([\w\s]+) : 재료 이름 그룹(알파벳과 숫자, 공백, + 패턴 한 번 이상 반복)
    # \( : 괄호 
    # ([\d\.]+[k]? : 재료 양 그룹(숫자.소숫점, k분자가 있을 경우 고려_kg)
    # g)\) : 'g' 다음 괄호 ) 
    ingredients = re.findall(r'([\w\s]+)\(([\d\.]+[k]?g)\)', str(parts))
    
    # 재료 이름 - 튜플의 첫번째요소(재료)에서 앞 뒤 공백 제거
    ingredient_names = [ingredient[0].strip() for ingredient in ingredients]
    # 재료 양 - 튜플의 두번째요소 양 저장
    ingredient_amount = [ingredient[1] for ingredient in ingredients]
    
    return ingredient_names, ingredient_amount
    

def get_recipe_data(recipe_name, data_dict):
    for recipe in data_dict:
        if recipe['RCP_NM'] == recipe_name:
            ingredient_names, ingredient_amount = extract_ingredients(recipe)
            
            # 딕셔너리로 구성하여 정리된 데이터 목록에 추가
            recipe_data = {
                'menu': recipe['RCP_NM'],
                'rcp': ingredient_names,
                'amount': ingredient_amount,
                'eng': recipe['INFO_ENG'],
                'car': recipe['INFO_CAR'],
                'pro': recipe['INFO_PRO'],
                'fat': recipe['INFO_FAT']
            }
            
            return recipe_data
    
    # 정규 표현식으로 한 거 중에 없으면 None
    return None
        

food_name = []
result = []

# 제목만 추출
for i in data_dict:
    name = i.get('RCP_NM')
    food_name.append(name)
    
for recipe_name in food_name :
    # 정리된 레시피 제목과 정보 추출
    result_data = get_recipe_data(recipe_name, data_dict)
    result.append(result_data)
    
print(result[:3])

[{'menu': '새우 두부 계란찜', 'rcp': [], 'amount': [], 'eng': 220.0, 'car': 3.0, 'pro': 14.0, 'fat': 17.0}, {'menu': '부추 콩가루 찜', 'rcp': [], 'amount': [], 'eng': 215.0, 'car': 20.0, 'pro': 14.0, 'fat': 9.0}, {'menu': '방울토마토 소박이', 'rcp': [], 'amount': [], 'eng': 45.0, 'car': 9.0, 'pro': 2.0, 'fat': 1.0}]


In [38]:
df11 = pd.DataFrame(result)
df11.tail(5)

,menu,rcp,amount,eng,car,pro,fat
995,삼색 머핀,"[당근, 시금치, 사과, 호두, 크랜베리, 박력분, 꿀, 버터, 소금, 베이킹소다,...","[10g, 10g, 10g, 5g, 5g, 150g, 15g, 90g, 1.5g, ...",651.3,82.6,9.5,31.4
996,수삼 냉채,[],[],38.4,9.6,1.4,0.2
997,아시아식 채소 두부 볶음,[],[],375.4,20.4,13.6,17.5
998,양배추 깻잎 피클,[],[],14.1,3.7,0.3,0.0
999,얼갈이 무침,[],[],71.7,6.8,4.7,4.1


In [45]:
cleaned_data  = []

for item in data_dict:
    parts_details = item['RCP_PARTS_DTLS']
    dish_name = item['RCP_NM']

    # 정규 표현식을 사용하여 재료와 양 추출
    # \b\w+ : 단어 앞뒤 지정으로 단어 의미
    # \b\s\d+ : 단어앞뒤 지정하고 공백과 한 개 이상 숫자
    # g(?:\(\d/\d\))? : g 포함해서 숫자/숫자와 형식 가능
    ingredients_info = re.findall(r'\b\w+\b\s\d+g(?:\(\d/\d\))?', str(parts_details))
    ingredient_name_list = []
    amount_list = []
    
    #print(ingredients_info) 
    # 재료와 양 추출해서 재료와 양 리스트에 넣기
    for ingredient_info in ingredients_info:
        # 단어 앞뒤+ 경계
        ingredient_name, amount = re.findall(r'\b\w+\b', ingredient_info)[:2]
        ingredient_name_list.append(ingredient_name+ ' ')
        amount_list.append(amount+ ' ')
        #print(ingredient_info)
        
    # 영양 정보 추출
    eng_info = item['INFO_ENG']
    car_info = item['INFO_CAR']
    pro_info = item['INFO_PRO']
    fat_info = item['INFO_FAT']

    # 딕셔너리로 구성하여 정리된 데이터 목록에 추가
    dish_dict = {
        'menu': dish_name,
        'rcp': ingredient_name_list,
        'amount' : amount_list,
        'eng': eng_info,
        'car': car_info,
        'pro': pro_info,
        'fat': fat_info
    }
    cleaned_data.append(dish_dict)

print(cleaned_data[:3])

[{'menu': '새우 두부 계란찜', 'rcp': ['연두부 ', '칵테일새우 ', '달걀 ', '생크림 ', '설탕 ', '무염버터 ', '시금치 '], 'amount': ['75g ', '20g ', '30g ', '13g ', '5g ', '5g ', '10g '], 'eng': 220.0, 'car': 3.0, 'pro': 14.0, 'fat': 17.0}, {'menu': '부추 콩가루 찜', 'rcp': ['조선부추 ', '날콩가루 ', '저염간장 ', '대파 ', '마늘 ', '고춧가루 ', '요리당 ', '참기름 '], 'amount': ['50g ', '7g ', '3g ', '5g ', '2g ', '2g ', '2g ', '2g '], 'eng': 215.0, 'car': 20.0, 'pro': 14.0, 'fat': 9.0}, {'menu': '방울토마토 소박이', 'rcp': ['방울토마토 ', '양파 ', '부추 ', '고춧가루 ', '멸치액젓 ', '매실액 ', '설탕 '], 'amount': ['150g ', '10g ', '10g ', '4g ', '3g ', '2g ', '2g '], 'eng': 45.0, 'car': 9.0, 'pro': 2.0, 'fat': 1.0}]


In [48]:
total_data = cleaned_data + result
total_data[:3]

[{'menu': '새우 두부 계란찜',
  'rcp': ['연두부 ', '칵테일새우 ', '달걀 ', '생크림 ', '설탕 ', '무염버터 ', '시금치 '],
  'amount': ['75g ', '20g ', '30g ', '13g ', '5g ', '5g ', '10g '],
  'eng': 220.0,
  'car': 3.0,
  'pro': 14.0,
  'fat': 17.0},
 {'menu': '부추 콩가루 찜',
  'rcp': ['조선부추 ', '날콩가루 ', '저염간장 ', '대파 ', '마늘 ', '고춧가루 ', '요리당 ', '참기름 '],
  'amount': ['50g ', '7g ', '3g ', '5g ', '2g ', '2g ', '2g ', '2g '],
  'eng': 215.0,
  'car': 20.0,
  'pro': 14.0,
  'fat': 9.0},
 {'menu': '방울토마토 소박이',
  'rcp': ['방울토마토 ', '양파 ', '부추 ', '고춧가루 ', '멸치액젓 ', '매실액 ', '설탕 '],
  'amount': ['150g ', '10g ', '10g ', '4g ', '3g ', '2g ', '2g '],
  'eng': 45.0,
  'car': 9.0,
  'pro': 2.0,
  'fat': 1.0}]

In [53]:
# 정규화로 레시피와 양 나눈 2개의 리스트를 묶어서 [] 채우기

merged_data = {}

for num in total_data:
    menu_name = num['menu']
    
    # menu_name에 merged_data의 키가 존재하지 않으면
    if menu_name not in merged_data:
        #merged 데이터에 추가
        merged_data[menu_name] = {'menu': menu_name, 'rcp': [], 'amount': [], 'eng': entry['eng'], 'car': entry['car'], 'pro': entry['pro'], 'fat': entry['fat']}
    
    # 키 존재하면 리스트에 재료, 양 추가 확장
    merged_data[menu_name]['rcp'].extend(num['rcp'])
    merged_data[menu_name]['amount'].extend(num['amount'])
    
    #앞에 '재료' 있으면 ''로 변경
    merged_data[menu_name]['rcp'] = [item.replace('재료 ', '') for item in merged_data[menu_name]['rcp']]

# merged_data에 저장된 값만 추출
merged_list = list(merged_data.values())

print(merged_list[:3])

[{'menu': '새우 두부 계란찜', 'rcp': ['연두부 ', '칵테일새우 ', '달걀 ', '생크림 ', '설탕 ', '무염버터 ', '시금치 '], 'amount': ['75g ', '20g ', '30g ', '13g ', '5g ', '5g ', '10g '], 'eng': 71.7, 'car': 6.8, 'pro': 4.7, 'fat': 4.1}, {'menu': '부추 콩가루 찜', 'rcp': ['조선부추 ', '날콩가루 ', '저염간장 ', '대파 ', '마늘 ', '고춧가루 ', '요리당 ', '참기름 '], 'amount': ['50g ', '7g ', '3g ', '5g ', '2g ', '2g ', '2g ', '2g '], 'eng': 71.7, 'car': 6.8, 'pro': 4.7, 'fat': 4.1}, {'menu': '방울토마토 소박이', 'rcp': ['방울토마토 ', '양파 ', '부추 ', '고춧가루 ', '멸치액젓 ', '매실액 ', '설탕 '], 'amount': ['150g ', '10g ', '10g ', '4g ', '3g ', '2g ', '2g '], 'eng': 71.7, 'car': 6.8, 'pro': 4.7, 'fat': 4.1}]


In [54]:
all_result = pd.DataFrame(merged_list)
all_result.head()

,menu,rcp,amount,eng,car,pro,fat
0,새우 두부 계란찜,"[연두부 , 칵테일새우 , 달걀 , 생크림 , 설탕 , 무염버터 , 시금치 ]","[75g , 20g , 30g , 13g , 5g , 5g , 10g ]",71.7,6.8,4.7,4.1
1,부추 콩가루 찜,"[조선부추 , 날콩가루 , 저염간장 , 대파 , 마늘 , 고춧가루 , 요리당 , 참...","[50g , 7g , 3g , 5g , 2g , 2g , 2g , 2g ]",71.7,6.8,4.7,4.1
2,방울토마토 소박이,"[방울토마토 , 양파 , 부추 , 고춧가루 , 멸치액젓 , 매실액 , 설탕 ]","[150g , 10g , 10g , 4g , 3g , 2g , 2g ]",71.7,6.8,4.7,4.1
3,순두부 사과 소스 오이무침,"[오이 , 땅콩 , 순두부 , 사과 ]","[70g , 10g , 40g , 50g ]",71.7,6.8,4.7,4.1
4,사과 새우 북엇국,"[북어채 , 새우 , 사과 , 양파 , 표고버섯 ]","[25g , 10g , 30g , 40g , 20g ]",71.7,6.8,4.7,4.1


In [55]:
df5.to_excel('all_result.xlsx')